In [1]:
%pip install openai pydantic jinja2 python_dotenv ipywidgets d20

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import re
import tomllib
import functools
import random
import d20

from jinja2 import Template
from pydantic import BaseModel
from openai import OpenAI, AzureOpenAI
from dotenv import load_dotenv
from d20 import roll

import ipywidgets as widgets
from IPython.display import display, clear_output

from typing import Literal, Union

load_dotenv()

with open("data/fight_club_2.toml", "rb") as f:
    config = tomllib.load(f)

message_pattern = re.compile(r"^([\w_ *]+)\:[ \s]*(.+)*$")
class Message(BaseModel):
    # Message from chatbot
    role: Literal["system", "assistant", "user"]
    content: str   

    # parse the content into dictionary
    def dict(self) -> dict:
        current_key = None
        parsed = {}
        lines = self.content.splitlines()
        # parse the content line by line
        for line in lines:
            res = message_pattern.search(line)
            if res is not None:
                current_key = res.group(1)
                if res.group(2) is not None:
                    parsed[current_key] = res.group(2).strip()
                else:
                    parsed[current_key] = ""
            elif current_key is not None and line.strip() != "":
                parsed[current_key] += "\n" + line.strip()
        return parsed

class Chapter(BaseModel):
    id: str
    next: str|None = None
    max_turns: int
    backgrounds: list[str]
    rules: list[str]
    initial: str
    
    def __init__(self, **kargs):
        super().__init__(**kargs)
    
    def get_messages(self) -> list[Message]:
        rendered_rules = render_rules(self.rules)
        system_content = Template(system).render({"rules": rendered_rules, "backgrounds":self.backgrounds, "character": character})
        initial_content = Template(self.initial).render({"rules": self.rules, "character": character})
        return [
            Message(role="system", content=system_content),
            Message(role="assistant", content=initial_content)
        ]

version = config["version"] # version of the config
system = config["system"] # system prompt template message
character = config["character"] # character information
chapters = config["chapters"] # chapters list
item_list = config["list"]["items"] # items list
enemy_list = config["list"]["enemies"] # items list

messages:list[Message] =[] # chat messages

tags = [] # tags list
items = character["items"] # items list

print(f"Version: {version}, Chapters: {len(chapters)}")

# find the chapter by id
def getChapterById(id:str) -> Chapter:
    for r in chapters:
        if r["id"] == id:
            return Chapter(**r) 
    return None

# next chapter find and initialize
def choose_next_chapter():
    global current_chapter, messages
    if current_chapter.next is not None:
        next = Template(current_chapter.next).render({"items": items, "tags": tags})
        print("Next round: ", next)
        next_chapter = getChapterById(next)
        if next_chapter is not None:
            current_chapter = next_chapter
            initials = current_chapter.get_messages()
            print(initials[1].content)
            show_controls(initials[1])
            messages = []
        else:
            print(f"Chapter '{next}' not found.")
    else:
        print("No next chapter found.")

# cout the messages by role, used for chapter's max_turns check
def count_messages(messages, role):
    return sum(1 for message in messages if message.role == role)

# chat with Azure OpenAI
def chat_with_azure(messages:list[Message], on_message: Union[callable, None] = None) -> Message:
    client = AzureOpenAI(api_key=os.environ.get("AZURE_OPENAI_API_KEY"), 
                         azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
                         api_version=os.environ.get("AZURE_OPENAI_API_VERSION")
                         )
    stream = client.chat.completions.create(
        model=os.environ.get("AZURE_OPENAI_MODEL_NAME"),
        messages=messages,
        stream=True,
        max_tokens=4000,
        temperature=float(os.environ.get("OPENAI_TEMPERATURE"))
    )

    generated = ""
    for chunk in stream:
        if len(chunk.choices) > 0 and chunk.choices[0].delta.content is not None:
            delta = chunk.choices[0].delta.content
            if (on_message is not None):
                on_message(chunk.choices[0].delta.content)
            generated += delta
    return Message(role="assistant", content=generated)

def sanity_check(current_sanity, difficulty=1):
    """
    Perform a sanity check in Call of Cthulhu.

    Parameters:
    current_sanity (int): The current sanity level of the character.
    difficulty (int): The difficulty level of the sanity check. Defaults to 1.

    Returns:
    dict: The result of the sanity check, including new sanity level and madness if any.
    """
    # Define a list of bouts of madness
    madness_list = [
        "Katsaridaphobia, Hallucination",
        "Cynophobia, Hallucination",
        "Hysterical, Screaming",
        "Paranoid, Conspiracy",
        "Rhinotillexomania, Compulsion",
        "Katsaridaphobia, Hallucination",
        "Haemaphobia, Hallucination",
        "Teratophobia, Hallucination",
        "Coulrophobia, Hallucination",
        "Paranoid, Gaslighting",
        "Hyterical, Biting Fingers",
        "Hysterical, Scratching",

    ]
    
    result = {
        'success': None,
        'roll': None,
        'threshold': None,
        'sanity_loss': 0,
        'new_sanity': current_sanity,
        'madness': None
    }
    
    # Roll a 100-sided die (percentile roll)
    roll = random.randint(1, 100)
    result['roll'] = roll

    # Calculate the effective sanity threshold
    threshold = max(1, min(99, current_sanity - (difficulty - 1) * 5))
    result['threshold'] = threshold

    # Determine the result of the sanity check
    if roll <= threshold:
        result['success'] = True
    else:
        result['success'] = False
        # Determine the sanity loss (for simplicity, a fixed amount is subtracted)
        sanity_loss = random.randint(6, 14)  # Roll a 1d10 for sanity loss
        result['sanity_loss'] = sanity_loss
        new_sanity = current_sanity - sanity_loss
        result['new_sanity'] = new_sanity
        
        if sanity_loss >= 5:
            # Assign a random bout of madness
            madness = random.choice(madness_list)
            result['madness'] = madness
    
    return result

current_sanity = character["stats"]["sanity"]
difficulty = 3

result = sanity_check(current_sanity, difficulty)

# do chat with the chatbot, and show the message with control buttons...
def do_chat(content: str = None, on_message: Union[callable, None] = None):
    if content is not None:
        turn = count_messages(messages, 'user') + 1
        if turn >= current_chapter.max_turns:
            content += f"\nwhisper: MAX_TURNS is reached."
        messages.append(Message(role="user", content=content))
        print(content)

    # change it to chat_with_azure to use Azure OpenAI
    concat = current_chapter.get_messages() + messages
    msg = chat_with_azure(concat, on_message)

    # print all the messages:
    show_controls(msg)
    messages.append(msg)

# render each rule from the chapter
def render_rules(rules:list[str]) -> str:
    lists = []
    for rule in rules:
        res = Template(rule).render({"character": character, "tags":tags, "items": items})
        if res is not None and res != "":
            lists.append(res)
    return lists

# handle player's action
def on_action(b, index:int):
    user_msg = f"I select '{index}'."
    do_chat(user_msg, lambda x: print(x, end=""))

# handle player's skill check
def on_skill(b, skill:str, difficulty:str):
    roll_num = roll("1d100").total
    if roll_num < 5:
        user_msg = f"i roll the dice for '{roll_num}' and get a CRITICAL SUCCESS!"
        do_chat(user_msg, lambda x: print(x, end=""))
        return
    if roll_num >= 95:
        user_msg = f"i roll the dice for '{roll_num}' and get a CRITICAL FAILURE!"
        do_chat(user_msg, lambda x: print(x, end=""))
        return
    
    difficulty_num = 15 if skill not in character["skills"] else character["skills"][skill]
    if difficulty == "hard":
        difficulty_num = int(difficulty_num * 0.5)
    if difficulty == "extreme":
        difficulty_num = int(difficulty_num * 0.2)
    
    print(f"rolled {roll_num} against {difficulty_num} on skill '{skill}'...")
    
    if roll_num <= difficulty_num:
        user_msg = f"i roll the dice for '{skill.upper()}' and get a success!"
    else:
        user_msg = f"i roll the dice for '{skill.upper()}' and get a failure."

    do_chat(user_msg, lambda x: print(x, end=""))

    difficulty_map = {"normal":50, "hard":25, "extreme": 10}
    success = roll_num <= character["skills"][skill] - difficulty_map[difficulty]
    user_msg = f"i roll the dice for '{roll_num}' and get a {'SUCCESS' if success else 'FAILURE'}!"
    do_chat(user_msg, lambda x: print(x, end=""))

# handle sanity check
def on_sanity_check(b, sanity:str, difficulty:str):
    global character
    result = sanity_check(character["stats"]["sanity"])
    if result['success']:
        msg = f"Sanity check SUCCESS! Roll: {result['roll']} <= Threshold: {result['threshold']}. No sanity loss."
        do_chat(msg, lambda x: print(x, end=""))
    else:
        msg = f"Sanity check FAILURE! Roll: {result['roll']} > Threshold: {result['threshold']}. Sanity loss: {result['sanity_loss']}. New sanity: {result['new_sanity']}."
        character["stats"]["sanity"] = result['new_sanity']
        if result['madness']:
            msg += f" Madness: {result['madness']}"
            do_chat(msg, lambda x: print(x, end=""))


            # Whisper the bout of madness
            whisper_msg = f"whisper: {result['madness']}."
            do_chat(whisper_msg, lambda x: print(x, end=""))

    do_chat(msg, lambda x: print(x, end=""))


# handle continue
def on_continue(b):
    user_msg = f"Continue..."
    do_chat(user_msg, lambda x: print(x, end=""))

# handle chapter's ending, and show next chapter
def on_next(b):
    choose_next_chapter()
        
# handle player's retry, it will remove the last assistant message,
# and re-ask the chatbot
def on_retry(b):
    # delete last assistant message
    messages.pop()
    do_chat(None, lambda x: print(x, end=""))

# handle player's item usage
def on_item(b, item:dict):
    user_msg = Template(item["description"]).render({"character": character, "tags":tags, "items": items})
    user_msg = f"I use '{item['id']}' - {user_msg}"
    do_chat(user_msg, lambda x: print(x, end=""))

# handle player's attack
def on_attack(b, target:dict):
    # player attack target
    hit = roll("1d6").total
    target["stats"]["hp"] -= hit

    if target["stats"]["hp"] <= 0:
        user_msg = f"I hit {target['name']}, and {target['name']} is dead..."
        do_chat(user_msg, lambda x: print(x, end=""))
        return

    user_msg = f"I hit {target['name']}, and {target['name']} lost some hp..."

    # target attack player
    hit = roll(target["stats"]["attack"]).total
    character["stats"]["hp"] -= hit

    if character["stats"]["hp"] <= 0:
        user_msg = f"I got hit by {target['name']}, and player is dead...\nWHISPER: GAME_OVER"
        do_chat(user_msg, lambda x: print(x, end=""))
        return

    user_msg += f"\nand I got hit by {target['name']}, lost some hp..."
    do_chat(user_msg, lambda x: print(x, end=""))



action_pattern = re.compile(r"(\d+). *(.+)")
# the main function to show the controls
def show_controls(msg:Message):
    dict = msg.dict()
    buttons = []

    if "choices" in dict.keys():
        # handle player's choices
        matches = action_pattern.finditer(dict["choices"])
        for match in matches:
            btn = widgets.Button(description=match.group(0), layout=widgets.Layout(width="fit-content"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill_check" in dict.keys() and "difficulty" in dict.keys():
        # handle skill check
        btn = widgets.Button(description=f"{dict['skill_check']} - {dict['difficulty']}", layout=widgets.Layout(width="fit-content"))
        btn.on_click(functools.partial(on_skill, skill=dict['skill_check'], difficulty=dict['difficulty']))
        buttons.append(btn)
    elif "sanity_check" in dict.keys() and "difficulty" in dict.keys():
        btn = widgets.Button(description=f"{dict['sanity_check']} - {dict['difficulty']}", layout=widgets.Layout(width="fit-content"))
        btn.on_click(functools.partial(on_sanity_check, sanity=dict['sanity_check'], difficulty=dict['difficulty']))
        buttons.append(btn)
    elif "battle_with" in dict.keys():
        enemy = next((x for x in enemy_list if x['id'] == dict["battle_with"]), None)
        # print("start battle with:", enemy.name)
        btn = widgets.Button(description=f"Attack", layout=widgets.Layout(width="fit-content"))
        btn.on_click(functools.partial(on_attack, target=enemy))
        buttons.append(btn)
        pass
    elif "ending" in dict.keys():
        # handle chapter ending, show next chapter button
        btn = widgets.Button(description=f"NEXT", layout=widgets.Layout(width="fit-content"))
        btn.on_click(on_next)
        buttons.append(btn)
    elif "game_over" in dict.keys():
        # handle game over
        print("\nGame Over - Hit 'Run All' button to restart again.")
    else:
        # If there is no choices or endings, show continue button
        btn = widgets.Button(description="Continue", layout=widgets.Layout(width="fit-content"))
        btn.on_click(on_continue)
        buttons.append(btn)
    
    # add tag from generated message
    if "add_tag" in dict.keys() and dict["add_tag"] not in tags:
        tags.append(dict["add_tag"])

    # remove tag from generated message
    if "remove_tag" in dict.keys() and dict["remove_tag"] in tags:
        tags.remove(dict["remove_tag"])
    
    # add item from generated message
    if "add_item" in dict.keys() and dict["add_item"] not in item_list:
        item = dict["add_item"]
        found = next((x for x in item_list if x['id'] == item), None)
        if (found is None):
            print(f"\nItem '{dict['add_item']}' not found.")
        else:
            items.append(item)
    
    # remove item from generated message
    if "remove_item" in dict.keys() and dict["remove_item"] in items:
        items.remove(dict["remove_item"])
    
    controls = []

    # retry buttons
    if len(messages) > 0:
        retry_btn = widgets.Button(description="Retry", layout=widgets.Layout(width="fit-content"))
        retry_btn.on_click(on_retry)
        controls.append(retry_btn)

    # show tag list
    if len(tags) > 0:
        tags_lbl = widgets.Label(value=f"Tags: {', '.join(tags)}")
        controls.append(tags_lbl)

    # inventory buttons
    inventory = [widgets.Label(value="Inventory:")]
    if len(items) > 0:
        for item in items:
            found = next((x for x in item_list if x['id'] == item), None)
            if found:
                item_btn = widgets.Button(description=found["id"], layout=widgets.Layout(width="fit-content"))
                item_btn.on_click(functools.partial(on_item, item=found))
                inventory.append(item_btn)
            else:
                print(f"\nItem '{item}' not found in the list.")
        controls.append(widgets.HBox(inventory))
    
    display(widgets.VBox(buttons))

    if (len(controls) > 0):
        display(widgets.HBox(controls))

current_chapter = getChapterById("r1")

Version: 0.0.3, Chapters: 7


In [ ]:
initials = current_chapter.get_messages()
print(initials[1].content)
show_controls(initials[1])

background: ground_floor
narration: 
You are Jack, a samurai warrior that has been transported into the future in the year 2678.

You arrived at the battle tower after hearing rumors that the fourth floor has portals that could transport you back into your timeline. As you enter the building, a battle robot stands between you and the stairs to the next floor. The robot's eyes glow red and is ready to fight.
choices:
1. Fight the robot
2. Convince the robot to let you pass
3. Run
4. Look for additional equipment nearby


I select '1'.
narration: 
You unsheathe your katana, ready to face the mechanical menace. The robot charges towards you with its metallic arms swinging wildly.

battle_with: robot

I hit super bot, and super bot lost some hp...
and I got hit by super bot, lost some hp...
narration: 
As your katana slices through the air, you manage to land a significant hit on the robot, causing sparks to fly from its chassis. However, the robot retaliates with a powerful punch that sends you staggering back.

sanity_check: sanity
difficulty: normal

Sanity check FAILURE! Roll: 67 > Threshold: 30. Sanity loss: 13. New sanity: 17. Madness: Katsaridaphobia, Hallucination
narration: 
The impact of the robot's punch not only wounds you physically but also shakes your mental resolve. You start to experience intense hallucinations, seeing giant cockroaches crawling over your skin, which isn't real. This phobia grips you as you struggle to focus on the real threat in front of you.

add_tag: katsaridaphobia
add_tag: hallucination

whisper: Katsaridaphobia, Hallucination.
narration: 
Your mind is in turmoil as you grapple with the hallucinations. The room seems to distort, and the robot in front of you intermittently transforms into a giant cockroach. Despite the confusion, you know you must continue the fight or perish.

choices:
1. Attempt to focus and strike the robot again.
2. Try to calm yourself and clear the hallucinations.
3. Use the environment to your advantage.
4. Retreat and find cover to regroup.

...

Sanity check FAILURE! Roll: 41 > Threshold: 5. Sanity loss: 11. New sanity: -6. Madness: Katsaridaphobia, Hallucination
narration: 
Your grip on reality slips further as the shadows morph and twist into ever more terrifying forms. The pain from your wounds and the overwhelming fear render you incapable of coherent thought or action. Your mind breaks under the strain, lost to the madness.

game_over: Your descent into madness is complete, and you are no longer able to continue the battle or ascend the tower.
Game Over - Hit 'Run All' button to restart again.


VBox()

whisper: Katsaridaphobia, Hallucination.
whisper: MAX_TURNS is reached.
narration: 
As the hallucinations consume your mind and the battle rages on, you lose track of time and purpose. The tower's challenges have proven too much, and your journey ends here in the clutches of madness and defeat.

game_over: The tower remains unconquered as your mind succumbs to the darkness.
Game Over - Hit 'Run All' button to restart again.


VBox()

Sanity check FAILURE! Roll: 41 > Threshold: 5. Sanity loss: 11. New sanity: -6. Madness: Katsaridaphobia, Hallucination
whisper: MAX_TURNS is reached.
narration: 
Your grip on reality slips further as the shadows morph and twist into ever more terrifying forms. The pain from your wounds and the overwhelming fear render you incapable of coherent thought or action. Your mind breaks under the strain, lost to the madness.

game_over: Your descent into madness